In [29]:
# 评论LDA主题分析与特征相关性研究
# 1. 对训练集/验证集/测试集的评论数据计算LDA主题
# 2. 分析评论的子评论数与各特征之间的相关性

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import re
import warnings
warnings.filterwarnings('ignore')

# 中文显示设置
plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

# 路径设置
ROOT_DIR = Path('D:/010_CodePrograms/L/LLM_su7')
RESULTS_DIR = ROOT_DIR / 'results'

print('库加载完成')

库加载完成


In [30]:
# 加载训练集、验证集、测试集
train_df = pd.read_pickle(ROOT_DIR / 'train.pkl')
val_df = pd.read_pickle(ROOT_DIR / 'val.pkl')
test_df = pd.read_pickle(ROOT_DIR / 'test.pkl')

print(f'训练集: {len(train_df):,} 条')
print(f'验证集: {len(val_df):,} 条')
print(f'测试集: {len(test_df):,} 条')
print(f'\n数据列: {train_df.columns.tolist()}')

训练集: 217,162 条
验证集: 27,145 条
测试集: 27,145 条

数据列: ['评论文案', '微博文案', '根评论文案', '父评论文案', '发布时间', '用户总转发数', '用户总评论数', '用户总点赞数', '用户是否认证', '是否一级评论', '子评论数', '点赞数', '微博转发数', '微博评论数', '微博点赞数']


In [31]:
# 合并所有数据用于LDA训练
train_df['数据集'] = 'train'
val_df['数据集'] = 'val'
test_df['数据集'] = 'test'

all_df = pd.concat([train_df, val_df, test_df], ignore_index=True)
print(f'合并后总数据量: {len(all_df):,} 条')

合并后总数据量: 271,452 条


In [32]:
# 安装必要的库
try:
    import jieba
    print('jieba已安装')
except ImportError:
    !pip install jieba -q
    import jieba
    print('jieba安装完成')

try:
    from gensim import corpora
    from gensim.models import LdaModel
    print('gensim已安装')
except ImportError:
    !pip install gensim -q
    from gensim import corpora
    from gensim.models import LdaModel
    print('gensim安装完成')

from tqdm import tqdm
tqdm.pandas()

jieba已安装
gensim已安装


In [33]:
# 停用词列表
STOPWORDS = set([
    '的', '了', '是', '在', '我', '有', '和', '就', '不', '人', '都', '一', '一个',
    '上', '也', '很', '到', '说', '要', '去', '你', '会', '着', '没有', '看', '好',
    '自己', '这', '那', '她', '他', '它', '们', '什么', '啊', '吗', '呢', '吧', '哈',
    '嗯', '哦', '呀', '啦', '嘛', '哎', '唉', '诶', '哇', '呵', '嘿', '噢', '喔',
    '么', '个', '为', '与', '或', '但', '而', '从', '被', '把', '让', '给', '对',
    '这个', '那个', '这样', '那样', '怎么', '如何', '为什么', '因为', '所以',
    '可以', '可能', '应该', '能', '想', '觉得', '知道', '感觉', '希望',
    '真的', '确实', '其实', '还是', '已经', '还', '再', '又', '太', '更', '最',
    '比较', '非常', '特别', '挺', '蛮', '相当', '十分', '极', '超', '真', '好像',
    '大家', '现在', '时候', '今天', '明天', '昨天', '这里', '那里', '哪里',
    '回复', '转发', '评论', '赞', '收藏', '分享', '@', '#', '…', '。', '，', '！',
    '？', '：', '；', '"', '"', ''', ''', '（', '）', '【', '】', '、', '~', '·'
])

def preprocess_text(text):
    """文本预处理：分词并去除停用词"""
    if pd.isna(text) or not isinstance(text, str):
        return []
    
    # 去除表情符号中的文字部分
    text_clean = re.sub(r'\[.*?\]', '', text)
    # 去除URL
    text_clean = re.sub(r'http\S+|www\S+', '', text_clean)
    # 去除@用户
    text_clean = re.sub(r'@\S+', '', text_clean)
    
    # 分词
    words = jieba.lcut(text_clean)
    
    # 过滤：去除停用词、单字、纯数字
    words = [w for w in words if w not in STOPWORDS 
             and len(w) > 1 
             and not w.isdigit()
             and not re.match(r'^\W+$', w)]
    
    return words

# 测试预处理
sample_text = all_df['评论文案'].iloc[0]
print(f'原文: {sample_text}')
print(f'分词结果: {preprocess_text(sample_text)}')

原文: "🥀.爱意化成风声，替我转交给你" 𝓛𝓸𝓋𝓮 𝓉𝓊𝓇𝓃𝓈 𝒾𝓃𝓉𝓸 𝓌𝒾𝓃𝒹, 𝓰𝒾𝓋𝓮 𝒾𝓉 𝓉𝓸 𝓎𝓸𝓊 𝒻𝓸𝓇 𝓂𝓮.🥀
分词结果: ['爱意', '化成', '风声', '交给']


In [36]:
# 对所有评论进行分词
print('正在对评论进行分词...')
all_df['分词结果'] = all_df['评论文案'].progress_apply(preprocess_text)

# 检查分词结果
empty_count = (all_df['分词结果'].apply(len) == 0).sum()
print(f'\n分词完成，空分词结果数量: {empty_count:,}')

正在对评论进行分词...


  0%|                                                                                                                                             | 0/271452 [00:00<?, ?it/s]

  1%|█▎                                                                                                                             | 2785/271452 [00:00<00:09, 27793.17it/s]

  2%|██▌                                                                                                                            | 5565/271452 [00:00<00:09, 27252.89it/s]

  3%|███▉                                                                                                                           | 8291/271452 [00:00<00:12, 20502.10it/s]

...

 98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████  | 267166/271452 [00:17<00:00, 14207.48it/s]

 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋ | 268591/271452 [00:18<00:00, 5247.61it/s]

 99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎| 270032/271452 [00:18<00:00, 6467.64it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 271452/271452 [00:18<00:00, 14518.55it/s]


分词完成，空分词结果数量: 17,807


In [37]:
# 构建词典和语料库
from gensim import corpora
from gensim.models import LdaModel

# 过滤掉空分词的文档
valid_docs = all_df[all_df['分词结果'].apply(len) > 0]['分词结果'].tolist()
print(f'有效文档数量: {len(valid_docs):,}')

# 构建词典
print('\n构建词典...')
dictionary = corpora.Dictionary(valid_docs)

# 过滤极端词频的词
dictionary.filter_extremes(no_below=50, no_above=0.5, keep_n=10000)
print(f'词典大小: {len(dictionary)}')

有效文档数量: 253,645

构建词典...


词典大小: 3994


In [38]:
# 构建语料库
print('构建语料库...')
corpus = [dictionary.doc2bow(doc) for doc in tqdm(valid_docs, desc='构建语料库')]
print(f'语料库大小: {len(corpus):,}')

构建语料库...


构建语料库:   0%|                                                                                                                                 | 0/253645 [00:00<?, ?it/s]

构建语料库:  10%|███████████▌                                                                                                     | 25871/253645 [00:00<00:00, 257819.56it/s]

构建语料库:  22%|████████████████████████▊                                                                                        | 55770/253645 [00:00<00:00, 281815.76it/s]

构建语料库:  34%|██████████████████████████████████████▌                                                                          | 86517/253645 [00:00<00:00, 293476.03it/s]

...

构建语料库:  72%|████████████████████████████████████████████████████████████████████████████████▌                               | 182539/253645 [00:00<00:00, 310326.81it/s]

构建语料库:  84%|██████████████████████████████████████████████████████████████████████████████████████████████▎                 | 213570/253645 [00:00<00:00, 309212.48it/s]

构建语料库:  97%|████████████████████████████████████████████████████████████████████████████████████████████████████████████▌   | 245780/253645 [00:00<00:00, 312852.64it/s]

构建语料库: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 253645/253645 [00:00<00:00, 305635.75it/s]

语料库大小: 253,645


In [40]:
# 训练LDA模型
NUM_TOPICS = 10  # 主题数量

print(f'训练LDA模型 (主题数: {NUM_TOPICS})...')
lda_model = LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=NUM_TOPICS,
    random_state=42,
    passes=10,
    alpha='auto',
    eta='auto',
    chunksize=2000
)

print('LDA模型训练完成！')

训练LDA模型 (主题数: 10)...


LDA模型训练完成！


In [24]:
# 查看主题词
print('='*60)
print('LDA主题词')
print('='*60)

for idx, topic in lda_model.print_topics(num_words=10):
    print(f'\n主题 {idx}:')
    # 解析主题词
    words = re.findall(r'"([^"]+)"', topic)
    weights = re.findall(r'([0-9.]+)\*', topic)
    for w, wt in zip(words, weights):
        print(f'  {w}: {float(wt):.4f}')

LDA主题词

主题 0:
  电池: 0.0520
  第一: 0.0430
  看看: 0.0380
  时间: 0.0340
  视频: 0.0340
  的话: 0.0280
  比亚迪: 0.0220
  为啥: 0.0220
  起来: 0.0160
  别的: 0.0140

主题 1:
  这么: 0.0820
  图片: 0.0460
  查看: 0.0300
  官方: 0.0220
  有点: 0.0210
  事情: 0.0210
  微博: 0.0200
  有人: 0.0190
  车子: 0.0190
  难道: 0.0160

主题 2:
  小米: 0.2110
  就是: 0.0920
  汽车: 0.0350
  雷军: 0.0210
  手机: 0.0210
  流量: 0.0170
  其他: 0.0160
  这些: 0.0160
  品牌: 0.0150
  营销: 0.0130

主题 3:
  问题: 0.0680
  事故: 0.0610
  车主: 0.0440
  不会: 0.0360
  这种: 0.0360
  出来: 0.0320
  一样: 0.0230
  那么: 0.0220
  一下: 0.0210
  还有: 0.0210

主题 4:
  不是: 0.0520
  智驾: 0.0440
  驾驶: 0.0440
  开车: 0.0220
  不能: 0.0200
  辅助: 0.0190
  安全: 0.0190
  如果: 0.0180
  高速: 0.0180
  方向盘: 0.0140

主题 5:
  速度: 0.0400
  但是: 0.0390
  车门: 0.0350
  他们: 0.0310
  直接: 0.0270
  打不开: 0.0240
  碰撞: 0.0220
  打开: 0.0150
  关系: 0.0140
  时速: 0.0140

主题 6:
  不要: 0.0410
  出事: 0.0340
  看到: 0.0330
  米粉: 0.0310
  技术: 0.0270
  我们: 0.0250
  生命: 0.0230
  不行: 0.0210
  企业: 0.0190
  男朋友: 0.0180

主题 7:
  车企: 0.0460
  家属: 0.04

In [42]:
# 为每条评论分配主题
def get_topic_distribution(doc_words):
    """获取文档的主题分布"""
    if len(doc_words) == 0:
        return [-1] + [0.0] * NUM_TOPICS  # 空文档返回-1主题
    
    bow = dictionary.doc2bow(doc_words)
    if len(bow) == 0:
        return [-1] + [0.0] * NUM_TOPICS
    
    topic_dist = lda_model.get_document_topics(bow, minimum_probability=0.0)
    
    # 获取主要主题
    main_topic = max(topic_dist, key=lambda x: x[1])[0]
    
    # 获取各主题概率
    probs = [0.0] * NUM_TOPICS
    for topic_id, prob in topic_dist:
        probs[topic_id] = prob
    
    return [main_topic] + probs

print('为每条评论分配主题...')
topic_results = all_df['分词结果'].progress_apply(get_topic_distribution)

# 拆分结果
all_df['主题'] = topic_results.apply(lambda x: x[0])
for i in range(NUM_TOPICS):
    all_df[f'主题{i}概率'] = topic_results.apply(lambda x: x[i+1])

print(f'\n主题分布统计:')
print(all_df['主题'].value_counts().sort_index())

为每条评论分配主题...


  0%|                                                                                                                                             | 0/271452 [00:00<?, ?it/s]

  2%|██▏                                                                                                                            | 4618/271452 [00:00<00:05, 45796.25it/s]

  3%|████▎                                                                                                                          | 9198/271452 [00:00<00:06, 39961.51it/s]

  5%|██████▏                                                                                                                       | 13241/271452 [00:00<00:06, 38275.32it/s]

...

 96%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████     | 260817/271452 [00:07<00:00, 36243.67it/s]

 97%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊   | 264442/271452 [00:07<00:00, 36239.33it/s]

 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍ | 268067/271452 [00:07<00:00, 36200.77it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 271452/271452 [00:07<00:00, 34573.15it/s]


主题分布统计:
主题
-1     30520
 0      1182
 1      1357
 2     16251
 3     14525
 4    198011
 5      2556
 6      1414
 7      2358
 8      1819
 9      1459
Name: count, dtype: int64


In [43]:
# 小米汽车相关词汇
XIAOMI_CAR_WORDS = [
    # 品牌/车型
    '小米', 'su7', 'SU7', '雷军', '小米汽车', '米车',
    # 竞品
    '特斯拉', 'model', 'Model', '比亚迪', '蔚来', '小鹏', '理想', '华为', '问界',
    '保时捷', 'taycan', 'Taycan', '极氪', '智己', '阿维塔',
    # 技术/配置
    '电池', '续航', '充电', '快充', '超充', '电机', '马力', '扭矩', '零百',
    '智驾', '自动驾驶', '辅助驾驶', 'noa', 'NOA', '激光雷达', '芯片',
    '底盘', '悬挂', '空悬', '轮毂', '卡钳', '刹车', 'brembo',
    # 外观/内饰
    '颜值', '外观', '内饰', '座椅', '大屏', '中控', '方向盘', '天幕',
    # 价格/购买
    '价格', '售价', '预售', '定金', '大定', '锁单', '提车', '交付', '产能',
    # 评价词
    '性价比', '配置', '堆料', '诚意', '良心', '黑科技'
]

def extract_features(row):
    """提取评论的各种特征"""
    text = row['评论文案'] if pd.notna(row['评论文案']) else ''
    
    features = {}
    
    # 1. 评论长度
    features['评论长度'] = len(text)
    
    # 2. 感叹号数量
    features['感叹号数'] = text.count('!') + text.count('！')
    
    # 3. 问号数量
    features['问号数'] = text.count('?') + text.count('？')
    
    # 4. 表情数量 (微博表情格式 [xxx])
    features['表情数'] = len(re.findall(r'\[.*?\]', text))
    
    # 5. 话题标签 (#xxx#)
    hashtags = re.findall(r'#([^#]+)#', text)
    features['话题标签数'] = len(hashtags)
    features['有话题标签'] = 1 if len(hashtags) > 0 else 0
    
    # 6. @用户数量
    features['@用户数'] = len(re.findall(r'@\S+', text))
    
    # 7. 小米汽车相关词汇数量
    xiaomi_count = 0
    text_lower = text.lower()
    for word in XIAOMI_CAR_WORDS:
        xiaomi_count += text_lower.count(word.lower())
    features['小米汽车词汇数'] = xiaomi_count
    features['含小米汽车词汇'] = 1 if xiaomi_count > 0 else 0
    
    # 8. 时间特征
    if pd.notna(row['发布时间']):
        dt = pd.to_datetime(row['发布时间'])
        features['发布小时'] = dt.hour
        features['发布星期'] = dt.dayofweek  # 0=周一
        features['是否工作日'] = 1 if dt.dayofweek < 5 else 0
    else:
        features['发布小时'] = -1
        features['发布星期'] = -1
        features['是否工作日'] = -1
    
    return pd.Series(features)

print('提取评论特征...')
feature_df = all_df.progress_apply(extract_features, axis=1)

# 合并特征
all_df = pd.concat([all_df, feature_df], axis=1)

print('\n特征提取完成！')
print(f'新增特征: {feature_df.columns.tolist()}')

提取评论特征...


  0%|                                                                                                                                             | 0/271452 [00:00<?, ?it/s]

  0%|                                                                                                                                  | 1/271452 [00:00<45:30:39,  1.66it/s]

  0%|▌                                                                                                                               | 1235/271452 [00:00<01:54, 2361.40it/s]

  1%|█▏                                                                                                                              | 2528/271452 [00:00<00:58, 4627.81it/s]

...

 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋ | 268671/271452 [00:28<00:00, 12504.34it/s]

 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎| 269990/271452 [00:28<00:00, 12676.44it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 271360/271452 [00:28<00:00, 12956.19it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 271452/271452 [00:37<00:00, 7208.87it/s]


特征提取完成！
新增特征: ['评论长度', '感叹号数', '问号数', '表情数', '话题标签数', '有话题标签', '@用户数', '小米汽车词汇数', '含小米汽车词汇', '发布小时', '发布星期', '是否工作日']


In [27]:
# 查看特征统计
feature_cols = ['评论长度', '感叹号数', '问号数', '表情数', '话题标签数', 
                '@用户数', '小米汽车词汇数', '发布小时', '发布星期']

print('特征统计:')
all_df[feature_cols].describe()

特征统计:


,评论长度,感叹号数,问号数,表情数,话题标签数,@用户数,小米汽车词汇数,发布小时,发布星期
count,271452.000000,271452.000000,271452.000000,271452.000000,271452.000000,271452.000000,271452.000000,271452.000000,271452.000000
mean,24.560921,0.071633,0.219472,0.244091,0.004763,0.019281,0.460943,14.013516,2.898535
std,24.981538,0.386786,0.649843,0.710019,0.105596,0.200727,1.010280,5.828456,1.712080
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,2.000000
50%,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.000000,3.000000
75%,31.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,19.000000,4.000000
max,227.000000,15.000000,17.000000,34.000000,20.000000,25.000000,25.000000,23.000000,6.000000


In [44]:
# 计算子评论数与各特征之间的相关性
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

# 定义要分析的特征列
feature_cols = [
    '评论长度', '感叹号数', '问号数', '表情数', '发布小时', '发布星期', 
    '是否工作日', '用户总评论数', '用户是否认证', '是否一级评论', 
    '话题标签数', '小米汽车词汇数', 'LDA主题'
]

# 计算Pearson相关系数
correlations = {}
for col in feature_cols:
    if col in all_df.columns:
        corr, pvalue = stats.pearsonr(all_df['子评论数'], all_df[col])
        correlations[col] = {'pearson_corr': corr, 'p_value': pvalue}

# 转换为DataFrame
corr_df = pd.DataFrame(correlations).T
corr_df = corr_df.sort_values('pearson_corr', ascending=False)
print("=" * 60)
print("子评论数与各特征的Pearson相关系数")
print("=" * 60)
print(corr_df.round(6).to_string())

子评论数与各特征的Pearson相关系数
         pearson_corr   p_value
是否一级评论       0.052414  0.000000
评论长度         0.023484  0.000000
小米汽车词汇数      0.017646  0.000000
是否工作日        0.013306  0.000000
表情数          0.012123  0.000000
问号数          0.008973  0.000003
发布小时         0.007207  0.000173
感叹号数        -0.000076  0.968538
用户是否认证      -0.000671  0.726514
用户总评论数      -0.001177  0.539810
话题标签数       -0.002003  0.296794
发布星期        -0.016789  0.000000


In [45]:
# 检查all_df所有列
print("all_df所有列:")
print(list(all_df.columns))
print(f"\n数据形状: {all_df.shape}")

all_df所有列:
['评论文案', '微博文案', '根评论文案', '父评论文案', '发布时间', '用户总转发数', '用户总评论数', '用户总点赞数', '用户是否认证', '是否一级评论', '子评论数', '点赞数', '微博转发数', '微博评论数', '微博点赞数', '数据集', '分词结果', '主题', '主题0概率', '主题1概率', '主题2概率', '主题3概率', '主题4概率', '主题5概率', '主题6概率', '主题7概率', '主题8概率', '主题9概率', '评论长度', '感叹号数', '问号数', '表情数', '话题标签数', '有话题标签', '@用户数', '小米汽车词汇数', '含小米汽车词汇', '发布小时', '发布星期', '是否工作日']

数据形状: (271452, 40)


In [46]:
# 补充：主题与子评论数的相关性
corr, pvalue = stats.pearsonr(all_df['子评论数'], all_df['主题'])
print(f"主题与子评论数相关系数: {corr:.6f}, p值: {pvalue:.6f}")

# 各主题概率与子评论数的相关性
print("\n各主题概率与子评论数的相关性:")
for i in range(10):
    col = f'主题{i}概率'
    corr, pvalue = stats.pearsonr(all_df['子评论数'], all_df[col])
    print(f"  {col}: {corr:.6f} (p={pvalue:.6f})")

主题与子评论数相关系数: 0.013328, p值: 0.000000

各主题概率与子评论数的相关性:
  主题0概率: 0.006080 (p=0.001536)
  主题1概率: 0.005040 (p=0.008639)
  主题2概率: 0.009031 (p=0.000003)
  主题3概率: 0.011794 (p=0.000000)
  主题4概率: 0.018784 (p=0.000000)
  主题5概率: 0.012332 (p=0.000000)
  主题6概率: 0.008593 (p=0.000008)
  主题7概率: 0.016539 (p=0.000000)
  主题8概率: 0.001877 (p=0.328073)
  主题9概率: 0.009485 (p=0.000001)


In [47]:
# 保存LDA模型和词典到results文件夹
import os
os.makedirs('../results', exist_ok=True)

# 保存LDA模型
lda_model.save('../results/lda_model')
dictionary.save('../results/lda_dictionary')
print("LDA模型已保存到 results/lda_model")
print("词典已保存到 results/lda_dictionary")

# 保存带LDA主题的数据到根目录（与train/val/test放在一起）
train_with_lda = all_df[all_df['数据集'] == 'train'].drop(columns=['数据集']).reset_index(drop=True)
val_with_lda = all_df[all_df['数据集'] == 'val'].drop(columns=['数据集']).reset_index(drop=True)
test_with_lda = all_df[all_df['数据集'] == 'test'].drop(columns=['数据集']).reset_index(drop=True)

train_with_lda.to_pickle('../train_with_lda.pkl')
val_with_lda.to_pickle('../val_with_lda.pkl')
test_with_lda.to_pickle('../test_with_lda.pkl')

print(f"\n数据已保存:")
print(f"  train_with_lda.pkl: {len(train_with_lda):,} 条")
print(f"  val_with_lda.pkl: {len(val_with_lda):,} 条")
print(f"  test_with_lda.pkl: {len(test_with_lda):,} 条")

LDA模型已保存到 results/lda_model
词典已保存到 results/lda_dictionary



数据已保存:
  train_with_lda.pkl: 217,162 条
  val_with_lda.pkl: 27,145 条
  test_with_lda.pkl: 27,145 条


In [48]:
# 删除之前保存的文件
import os
for f in ['../train_with_lda.pkl', '../val_with_lda.pkl', '../test_with_lda.pkl']:
    if os.path.exists(f):
        os.remove(f)
        print(f"已删除: {f}")

# 重新加载原始数据并添加序号列
train_df = pd.read_pickle('../train.pkl')
val_df = pd.read_pickle('../val.pkl')
test_df = pd.read_pickle('../test.pkl')

# 添加序号列
train_df.insert(0, '序号', range(len(train_df)))
val_df.insert(0, '序号', range(len(val_df)))
test_df.insert(0, '序号', range(len(test_df)))

# 保存回原文件
train_df.to_pickle('../train.pkl')
val_df.to_pickle('../val.pkl')
test_df.to_pickle('../test.pkl')

print("原文件已添加序号列并保存:")
print(f"  train.pkl: {len(train_df):,} 条, 列: {list(train_df.columns[:3])}...")
print(f"  val.pkl: {len(val_df):,} 条")
print(f"  test.pkl: {len(test_df):,} 条")

已删除: ../train_with_lda.pkl
已删除: ../val_with_lda.pkl
已删除: ../test_with_lda.pkl


原文件已添加序号列并保存:
  train.pkl: 217,162 条, 列: ['序号', '评论文案', '微博文案']...
  val.pkl: 27,145 条
  test.pkl: 27,145 条


In [49]:
# 从all_df提取LDA主题，按数据集分开保存
# 注意：all_df中的顺序与原数据一致

train_mask = all_df['数据集'] == 'train'
val_mask = all_df['数据集'] == 'val'
test_mask = all_df['数据集'] == 'test'

# 创建LDA数据（序号 + 主题）
train_lda = pd.DataFrame({
    '序号': range(train_mask.sum()),
    '主题': all_df.loc[train_mask, '主题'].values
})
val_lda = pd.DataFrame({
    '序号': range(val_mask.sum()),
    '主题': all_df.loc[val_mask, '主题'].values
})
test_lda = pd.DataFrame({
    '序号': range(test_mask.sum()),
    '主题': all_df.loc[test_mask, '主题'].values
})

# 保存LDA文件
train_lda.to_pickle('../train_lda.pkl')
val_lda.to_pickle('../val_lda.pkl')
test_lda.to_pickle('../test_lda.pkl')

print("LDA主题文件已保存:")
print(f"  train_lda.pkl: {len(train_lda):,} 条")
print(f"  val_lda.pkl: {len(val_lda):,} 条")
print(f"  test_lda.pkl: {len(test_lda):,} 条")
print(f"\n列: {list(train_lda.columns)}")
print(f"\n示例:\n{train_lda.head()}")

LDA主题文件已保存:
  train_lda.pkl: 217,162 条
  val_lda.pkl: 27,145 条
  test_lda.pkl: 27,145 条

列: ['序号', '主题']

示例:
   序号  主题
0   0   4
1   1   4
2   2   4
3   3   8
4   4   4


In [51]:
# 给csv文件也添加序号列
train_csv = pd.read_csv('../train.csv')
val_csv = pd.read_csv('../val.csv')
test_csv = pd.read_csv('../test.csv')

# 添加序号列
train_csv.insert(0, '序号', range(len(train_csv)))
val_csv.insert(0, '序号', range(len(val_csv)))
test_csv.insert(0, '序号', range(len(test_csv)))

# 保存回原文件
train_csv.to_csv('../train.csv', index=False)
val_csv.to_csv('../val.csv', index=False)
test_csv.to_csv('../test.csv', index=False)

print("CSV文件已添加序号列并保存:")
print(f"  train.csv: {len(train_csv):,} 条")
print(f"  val.csv: {len(val_csv):,} 条")
print(f"  test.csv: {len(test_csv):,} 条")

CSV文件已添加序号列并保存:
  train.csv: 217,162 条
  val.csv: 27,145 条
  test.csv: 27,145 条


In [52]:
# 重新设置连续序号
train_len = 217162
val_len = 27145
test_len = 27145

train_start, train_end = 0, train_len
val_start, val_end = train_len, train_len + val_len
test_start, test_end = train_len + val_len, train_len + val_len + test_len

print(f"序号范围:")
print(f"  训练集: {train_start} ~ {train_end-1}")
print(f"  验证集: {val_start} ~ {val_end-1}")
print(f"  测试集: {test_start} ~ {test_end-1}")

# 修改pkl文件
train_df = pd.read_pickle('../train.pkl')
val_df = pd.read_pickle('../val.pkl')
test_df = pd.read_pickle('../test.pkl')

train_df['序号'] = range(train_start, train_end)
val_df['序号'] = range(val_start, val_end)
test_df['序号'] = range(test_start, test_end)

train_df.to_pickle('../train.pkl')
val_df.to_pickle('../val.pkl')
test_df.to_pickle('../test.pkl')

# 修改csv文件
train_csv = pd.read_csv('../train.csv')
val_csv = pd.read_csv('../val.csv')
test_csv = pd.read_csv('../test.csv')

train_csv['序号'] = range(train_start, train_end)
val_csv['序号'] = range(val_start, val_end)
test_csv['序号'] = range(test_start, test_end)

train_csv.to_csv('../train.csv', index=False)
val_csv.to_csv('../val.csv', index=False)
test_csv.to_csv('../test.csv', index=False)

# 修改lda文件
train_lda = pd.read_pickle('../train_lda.pkl')
val_lda = pd.read_pickle('../val_lda.pkl')
test_lda = pd.read_pickle('../test_lda.pkl')

train_lda['序号'] = range(train_start, train_end)
val_lda['序号'] = range(val_start, val_end)
test_lda['序号'] = range(test_start, test_end)

train_lda.to_pickle('../train_lda.pkl')
val_lda.to_pickle('../val_lda.pkl')
test_lda.to_pickle('../test_lda.pkl')

print("\n所有文件已更新为连续序号")

序号范围:
  训练集: 0 ~ 217161
  验证集: 217162 ~ 244306
  测试集: 244307 ~ 271451



所有文件已更新为连续序号
